In [1]:
import os
import geopandas as gpd
import ee
import geemap

In [2]:
## Register with ee
# ee.Authenticate(authorization_code='4/1AWgavdf9gch8us9pDJkVc_RGBg9LxSoiYkEcbEDEQwigeGYJPNGKjQLT9AI')
ee.Initialize()

In [3]:
## Create earthengine feature for BAWLD
gdf = gpd.read_file('/mnt/g/Other/Kuhn-olefeldt-BAWLD/BAWLD/BAWLD_V1___Shapefile.zip')
# bawld = geemap.shp_to_ee('/mnt/g/Other/Kuhn-olefeldt-BAWLD/BAWLD/BAWLD_V1___Shapefile.zip')
# bawld = geemap.gdf_to_ee(gdf)

ERROR 1: PROJ: proj_create_from_database: Open of /home/ekyzivat/mambaforge/envs/geospatial/share/proj failed


In [4]:
bawld_crs = gdf.crs.to_wkt()
gdf.crs

<Derived Projected CRS: PROJCS["Lambert_Azimuthal_Equal_Area",GEOGCS["WGS  ...>
Name: Lambert_Azimuthal_Equal_Area
Axis Info [cartesian]:
- [south]: Easting (metre)
- [south]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Lambert Azimuthal Equal Area
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [5]:
## Load BAWLD
bawld = ee.FeatureCollection('projects/ee-ekyzivat/assets/Shapes/BAWLD/bawld_grid')

In [6]:
## View
Map = geemap.Map()
Map.addLayer(bawld, {}, 'BAWLD grid')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [21]:
## Load GSW
gsw = ee.Image('JRC/GSW1_1/GlobalSurfaceWater')
occurrence = gsw.select('occurrence')

In [22]:
## Vis GSW
VIS_OCCURRENCE = {
  'min':0,
  'max':100,
  'palette': ['red', 'blue']
}
VIS_WATER_MASK = {
  'palette': ['white', 'black']
}

## Create a water mask layer, and set the image mask so that non-water areas are opaque.
water_mask = occurrence.gt(90).selfMask()

## Map Layers

Map.addLayer(occurrence.updateMask(occurrence.divide(100)), VIS_OCCURRENCE, "Water Occurrence (1984-2018)")
Map.addLayer(water_mask, VIS_WATER_MASK, '90% occurrence water mask', False)

## Try to filter features to reduce cost

In [17]:
## Filter BAWLD to reduce size of operation
# bawldF = bawld.filter("cell_ID >= 100 AND cell_id <=120")
# print(bawldF.first())

ee.Element({
  "functionInvocationValue": {
    "functionName": "Collection.first",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.filter",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.loadTable",
                "arguments": {
                  "tableId": {
                    "constantValue": "projects/ee-ekyzivat/assets/Shapes/BAWLD/bawld_grid"
                  }
                }
              }
            },
            "filter": {
              "constantValue": "cell_ID >= 100 AND cell_id <=120"
            }
          }
        }
      }
    }
  }
})


In [23]:
## Select spatially
roi = ee.FeatureCollection(ee.Geometry.Polygon(
    [[[-126.3, 57.5],
      [-124.8, 57.5],
      [-126.3, 56.7],
      [-124.8, 56.7]]]))
# roi = ee.FeatureCollection(ee.Geometry.Polygon([[[-26.3, 57.5]
#       [-124.8, 57.5],
#       [-26.3, 56.7],
#       [-124.8, 56.7]]]))
bawldF = bawld.filterBounds(roi)

In [13]:
## Select spatially
# roi = ee.FeatureCollection(Map.draw_features)
# bawldF = bawld.filterBounds(roi)

## Verify filter

In [24]:
Map2 = geemap.Map()
Map2.addLayer(bawldF, {}, 'BAWLD filtered')
Map2.centerObject(bawldF, 9);
Map2

Map(center=[57.26721506172541, -125.51036863401669], controls=(WidgetControl(options=['position', 'transparent…

## Stats

In [39]:
## Zonal stats
out_dir = '/mnt/g/Ch4/GSW_zonal_stats'
bawld_stats_pth = os.path.join(out_dir, 'bawld_stats.csv')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# statistics_type can be either 'SUM' or 'PERCENTAGE'
# denominator can be used to convert square meters to other areal units, such as square kilometers
geemap.zonal_statistics_by_group(
    occurrence,
    bawldF,
    bawld_stats_pth,
    statistics_type='SUM',
    denominator=1000000,
    decimal_places=3,
    crs=bawld_crs,
    tile_scale=1 # default is 1, but it exceeds GEE capacity
)

Computing ... 
Generating URL ...
Please wait ...
Data downloaded to /mnt/g/Ch4/GSW_zonal_stats/bawld_stats.csv


In [40]:
## View csv
!head {bawld_stats_pth}

Class_70,Class_71,Class_72,Class_73,Class_74,Class_75,Class_76,Class_77,Class_sum,Class_8,Class_78,Class_7,Class_79,Class_9,Class_0,Class_2,Class_1,Class_4,Class_3,Class_6,Class_5,Class_80,Class_60,Class_61,Class_62,Class_63,Class_64,Class_65,Class_66,Class_67,Class_68,Class_69,Class_92,Class_93,Class_94,Class_95,Class_96,Class_97,Class_10,Class_98,Class_11,Class_99,Class_12,Class_13,Class_14,Class_15,Class_16,Class_17,Class_18,Class_19,Class_81,Class_82,Class_83,Class_84,Class_85,Class_86,Class_87,Class_88,Class_89,Class_100,Class_90,Class_91,Class_30,Class_31,Class_32,Class_33,Class_34,Class_35,Class_36,Class_37,Class_38,Class_39,Class_20,Class_21,Class_22,Class_23,Class_24,Class_25,Class_26,Class_27,Class_28,Class_29,Class_50,Class_51,Class_52,Class_53,Class_54,Class_55,Class_56,Class_57,Class_58,Class_59,Class_40,Class_41,Class_42,Class_43,Class_44,Class_45,Class_46,Class_47,Class_48,Class_49,FEN_H,FEN_L,MPL,MYL_L,SPL,LAK_L,MYL_H,WTU_H,WTU_L,WTU,BOG,Area_Pct,BOG_L,MYL,BOR,BOG_H,ROC

Output when I tried to save as .shp (~7 min):
```python
Computing ... 
Generating URL ...
Downloading data from https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/2439eef1ef1e866e84ef7c39c7a25950-9928bfc2ae36fcea36b19e2b8f795f73:getFeatures
Please wait ...
An error occurred while downloading. 
 Retrying ...
Generating URL ...
Downloading data from https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/396259f2c3d332ef482a33098d6cc711-36e72eb94399a5afd8debbacfcaffc6c:getFeatures
Please wait ...
Output exceeds the size limit. Open the full output data in a text editor
---------------------------------------------------------------------------
BadZipFile                                Traceback (most recent call last)
File ~/mambaforge/envs/geospatial/lib/python3.10/site-packages/geemap/common.py:1579, in ee_export_vector(ee_object, filename, selectors, verbose, keep_zip, timeout, proxies)
   1578 if filetype == "shp":
-> 1579     with zipfile.ZipFile(filename) as z:
   1580         z.extractall(os.path.dirname(filename))

File ~/mambaforge/envs/geospatial/lib/python3.10/zipfile.py:1267, in ZipFile.__init__(self, file, mode, compression, allowZip64, compresslevel, strict_timestamps)
   1266 if mode == 'r':
-> 1267     self._RealGetContents()
   1268 elif mode in ('w', 'x'):
   1269     # set the modified flag so central directory gets written
   1270     # even if no files are added to the archive

File ~/mambaforge/envs/geospatial/lib/python3.10/zipfile.py:1334, in ZipFile._RealGetContents(self)
   1333 if not endrec:
-> 1334     raise BadZipFile("File is not a zip file")
   1335 if self.debug > 1:

BadZipFile: File is not a zip file

During handling of the above exception, another exception occurred:

ValueError                                Traceback (most recent call last)
File ~/mambaforge/envs/geospatial/lib/python3.10/site-packages/geemap/common.py:6960, in zonal_stats_by_group(in_value_raster, in_zone_vector, out_file_path, statistics_type, decimal_places, denominator, scale, crs, tile_scale, return_fc, verbose, timeout, proxies, **kwargs)
...
   6960         ee_export_vector(final_result, filename, timeout=timeout, proxies=proxies)
   6962 except Exception as e:
-> 6963     raise Exception(e)

Exception: File is not a zip file


In [22]:
geemap.ee_export_vector('https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/tables/396259f2c3d332ef482a33098d6cc711-36e72eb94399a5afd8debbacfcaffc6c:getFeatures', bawld_stats_pth, keep_zip=True)

ValueError: ee_object must be an ee.FeatureCollection

In [30]:
# geemap.datasets.get_geemap_data_list()
geemap.get_nb_template()

'/home/ekyzivat/mambaforge/envs/geospatial/lib/python3.10/site-packages/geemap/data/template/template.py'